## 6.2 기울기 소실과 LSTM
- 기울기 폭주는 Gradient Clipping 기법으로 해결가능 헀으나 기울기 손실을 위해서는 RNN의 구조 자체를 고쳐야하는 것이 정설이다.
- 핵심은 RNN에 Gate를 추가 하는 것이다.
- LSTM은 기울기 손실을 일으키지 않는다.

#### 6.2.1 LSTM의 인터페이스

![image](https://github.com/choibigo/Study/assets/38881179/ef7998bb-4ba3-4905-b66a-4ce7166b1f63)

- tanh(ht-1*Wh + xtWx + b) 계산을 tanh라는 직사각형 노드 하나로 표현한다.

![image](https://github.com/choibigo/Study/assets/38881179/784275ce-f540-4c1d-83f8-53eae858fd1c)

- LSTM 계층의 인터페이스에는 C라는 경로 가 있다는 차이가 있다.
- C를 Momory Cell 혹은 Cell 이라 하며, LSTM 전용의 기억 메커니즘 이다.
- Memory Cell의 특징은 데이터를 자기 자신으로만(LSTM 계층 내에서만) 주고 받는다는 것이다.
- LSTM 계층 내에서만 완결되고, 다른 계층으로는 출력하지않는다.
- LSTM을 출력을 받는 쪽에서 보면 ht 만 보인다, Memory Cell C는 외부에서는 보이지 않는다.

#### 6.2.2 LSTM 계층 조립하기
- LSTM에는 Memory Cell Ct가 있다.
- Ct에는 시각 t에서의 LSTM의 기억이 저장돼 있다.
- 과거로부터 시각 t까지에 필요한 모든 정보가 저장돼 있다고 가정한다.
- 필요한 정보를 모두 간직한 이 Memory Cell을 바탕으로 외부계층에 (혹은 다음 LSTM) ht를 출력한다, 이 외부로 출력되는 ht는 ct에 tanh 함수를 통과시킨 값이다.

![image](https://github.com/choibigo/Study/assets/38881179/5165661a-311e-49e1-9f7e-0bb55edc15ba)

- Memory Cell Ct를 바탕으로 은닉 상태 ht를 계산 한다.
- Memory Cell Ct는 (Ct-1, ht-1, xt)로부터 '어떤 계산'을 수행하여 구한다.
- ht = tanh(ct)인데 이는 ct의 각 원소에 tanh 함수를 적용한다는 뜻이다.


#### Gate란?
- 데이터의 흐름을 제어한다
- 데이터의 흐름을 멈추거나 배출하거나 정도를 조절할 수 있다.
- 게이트의 열림 상태를 제어하기 위해서 전용 가중치 매개변수를 이용하며, 이 가중치 매개변수는 학습 데이터로부터 갱신된다.

#### 6.2.3 output 게이트
- tanh(ct)의 각 원소에 대해 '그것이 다음 시각의 은닉 상태에 얼마나 중요한가'를 조정한다.
- 이 게이트는 다음 은닉상태 ht의 출력을 담당하는 게이트 이므로 Output Gate라고 한다.
- Output 게이트의 열림상태(다음 몇 %만 보낼까)는 입력 xt와 이전 상태 ht-1로부터 구한다, tanh(ct)의 값을 얼마나 output할지 결정한다.

![image](https://github.com/choibigo/Study/assets/38881179/f8e5cd94-7de1-4cca-b9b9-ef87ecd3fe90)

- xt에는 가중치 Wx(o)가, 이전 시각의 은닉상태 ht-1에는 가중치Wn(o)이 붙어 있다.
- 그리고 이 행렬의 곱과 편향 b(o)를 모두 더한 후 시그모이드 함수를 거쳐 출력 게이트의 출려 O를 구한다.
- 마지막으로 O와 tanh(Ct)의 원소별 곱을 ht로 출력한다

![image](https://github.com/choibigo/Study/assets/38881179/33e74b43-6c2c-45d3-96e2-1a44dffa903f)

- O와 tanh(ct)를 원소별 곱을 하여 원소마다 얼마나 반영할 지를 결정 한다, tanh이후 값을 얼마나 통과 시킬지 결정하므로 Gate라고 한다.

```
- tanh 의 출력은 -1 ~ 1의 실수이다.
- -1 ~ 1의 수치를 그 안에 인코딩된 '정보'의 정도를 표시한다고 해석할 수 있다.
- 시그모이드 함수는 0~1의 실수이다.
- 데이터를 얼만큼 통과시킬지를 결정하는 비율을 의미한다.
```

#### 6.2.4 forget 게이트
- 셀에 '무엇을 잊을까'를 명확하게 지시하는 것이다.
- ct-1의 기억중 불필요한 기억을 잊게 해주는 게이트를 forget gate라고 한다.

![image](https://github.com/choibigo/Study/assets/38881179/4474ff45-5eff-4815-891e-f725760377cf)

- forget 게이트 σ안에는 forget 게이트 전용의 가중치 매개변수가 있다.

![image](https://github.com/choibigo/Study/assets/38881179/d5c7ae52-2748-4bd7-abb9-35699bef313a)

- forget 게이트의 출력 f가 구해진다, 그리고 이 f와 이전 기억 셀인 ct-1과의 원소별 곱 으로 ct를 구한다.
- forget Gate를 위한 Wx, Wh 와 Output Gate를 위한 Wx, Wh가 존재 한다.

#### 6.2.5 새로운 기억셀
- forget gate를 거치면서 이전 시각의 기억 셀로부터 잊어야할 기억이 삭제 되었다.
- 그런데 이 상태로는 기억 셀이 잊는것 밖에 못한다, 그래서 새로 기억해야할 정보를 기억셀에 추가 해야한다.

![image](https://github.com/choibigo/Study/assets/38881179/2f694e95-d6e0-4f5f-a4ca-1f03627d6f9a)

- 그러기 위해서 tanh 노드를 추가한다.
- tanh 노드가 계산한 결과가 이전 시각의 기억세 ct-1에 더해진다.
- 기억 셀에 새로운 '정보가'추가 된 것이다.
- 이 tanh 노드는 '게이트'가 아니며, 새로운 '정보'를 기억 셀에 추가하는 것이 목적이다.
- 따라서 활성화 함수로는 시그모이드가 아닌 tanh 함수를 사용한다.

![image](https://github.com/choibigo/Study/assets/38881179/b4204fe9-4ce3-4ebb-87ea-b1d4f3c0d7cc)

- 기억셀에 추가 하는 새로운 기억을 g라고 표기 했다.
- 이 g가 이전 시각의 기억셀인 ct-1과 더해짐으로써 새로운 기억이 생겨난다.

#### 6.2.6 Input Gate

![image](https://github.com/choibigo/Study/assets/38881179/87f1623a-9a53-4e4e-ab3c-8bc440062939)

- input 게이트는 g의 각 원소가 새로 추가되는 정보로써의 가치가 얼마나 큰지를 판단 한다.
- 입력 데이터에 대해 gate를 적용하여 새로운 정보를 무비판적으로 수요하는 것이 아니라, 적절히 수용할 수 있게 해주는 게이트 이다.
- 또다르게 생각하면, Input Gate에 의해 가중된 정보가 새로 추가되는 셈이다.

![image](https://github.com/choibigo/Study/assets/38881179/50040206-f541-446d-b24b-0ed4d8676a27)

- i와 g를 원소별 곱을 하여 기억셀에 추가한다

#### 6.2.7 LSTM의 기울기 흐름
- 